In [98]:
n, m = 4, 6
rows = ['0 1 34 99', '1 2 1 3', '0 2 1 2']
#rows = ['0 1 1 1', '1 2 3 3', '0 2 2 1']

#rows = ['0 1 1 100', '1 2 96 10', '1 2 88 1']


import sys


n,m = [int(n),int(m)]

main_tree = dict()

def dict_key(u, v):
    str_u = str(u)
    str_v = str(v)
    return str_u + str_v if u < v else str_v + str_u

def gcd(a, b):
    if a == 0:
        return b
    else:
        return gcd(b % a, a)



def find_insertion_point(keys, value, start, end):
    #print('iter', keys, value, start, end)
    if start == end:
        return start
    middle = (start + end) // 2
    #print('middle', middle)
    m_frac = keys[middle][1]
    
    frac_value = m_frac[0] * value[1] - m_frac[1] * value [0]
    
    if frac_value == 0:
        if m_frac[0] > value[0]:
            return find_insertion_point(keys, value, middle + 1, end)
        else:
            return find_insertion_point(keys, value, start, middle)
    
    if frac_value < 0:
        #print('bigger')
        return find_insertion_point(keys, value, middle + 1, end)
    else:
        #print('lower')
        return find_insertion_point(keys, value, start, middle)

for row in rows:
    u,v,a,b = row.split(' ')
    u,v,a,b = [int(u),int(v),int(a),int(b)]
    
    # ignore self-loops
    if u == v:
        continue
    
    key = dict_key(u, v)

    if key not in main_tree:
        main_tree[key] = (a, b)
    else:
        frac = a * main_tree[key][1] - b * main_tree[key][0]
        if frac > 0 or (frac == 0 and a < main_tree[key][1]):
            main_tree[key] = (a, b)
        

sorted_keys = []
for key in main_tree:
    value = main_tree[key]
    if len(sorted_keys) == 0:
        sorted_keys.append((key, value))
    else:
        insertion_point = find_insertion_point(sorted_keys, value, 0, len(sorted_keys))
        sorted_keys.insert(insertion_point, (key, value))

print(sorted_keys)
        
i = len(sorted_keys) - 1
visited = set()
fraction = (0, 0)

while i >= 0:
#     if len(visited) == n:
#         break
    
    key, value = sorted_keys[i]
    #print(key, value, i)
    i -=1
    
    if key[0] in visited and key[1] in visited:
        #print('all visited', key, visited)
        continue
    else:
        fraction = (fraction[0] + value[0], fraction[1] + value[1])
        visited.add(key[0])
        visited.add(key[1])
    
print('' + str(fraction[0]) + '/' + str(fraction[1]))



[('12', (1, 3)), ('01', (34, 99)), ('02', (1, 2))]
35/101


## Semi-working version

In [138]:
n, m = 4, 6
rows = ['0 1 1 1', '1 2 1 1', '2 3 1 1', '3 0 1 1', '0 2 3 2', '1 3 3 2']
#rows = ['0 1 34 99', '0 1 35 98', '1 2 1 3', '1 1 1 3', '1 2 1 4', '1 2 1 2', '0 2 1 2', '0 3 34 99']
#rows = ['0 1 1 100', '1 2 96 10', '1 2 88 1']
import sys


n,m = [int(n),int(m)]

main_tree = dict()
# these are the keys that represent alternative ways to connect two edges
branching_keys = dict()

def dict_key(u, v):
    str_u = str(u)
    str_v = str(v)
    return str_u + str_v if u < v else str_v + str_u

def gcd(a, b):
    if a == 0:
        return b
    else:
        return gcd(b % a, a)
    
def is_fraction_greater(a, b):
    return a[1][0] / a[1][1] > b[1][0] / b[1][1]

for row in rows:
    u,v,a,b = row.split(' ')
    u,v,a,b = [int(u),int(v),int(a),int(b)]
    
    # ignore self-loops
    if u == v:
        continue
    
    key = dict_key(u, v)

    if (key in main_tree):
        if (key in branching_keys):
            branching_keys[key].append((a, b))
        else:
            branching_keys[key] = [main_tree[key], (a, b)]
    else:
        main_tree[key] = (a, b)

trees = []

for key, elems in branching_keys.items():
    i = 0

    while i < len(elems) - 1:
        new_dicts = []
        for tree_dict in trees:
            n_dict = tree_dict.copy()
            n_dict[key] = elems[i + 1]
            new_dicts.append(n_dict)
        trees += new_dicts
        i += 1
    
    if len(trees) == 0:
        for elem in elems:
            new_dict = main_tree.copy()
            new_dict[key] = elem
            trees.append(new_dict)
    
if len(trees) == 0:
    trees = [main_tree]

#print(trees)
    
max_tree_fraction = (None, (-1, 1))
    
for tree in trees:
    max_coeff = (None, -1)
    variations = []
    for key in tree:
        #print(key)
        if key[0] == '0':
            variations.append((key, tree[key]))
    #print(variations)
    
    for variation in variations:
        first_key = variation[0]
        visited = {first_key[0], first_key[1]}
        fraction = (first_key, tree[first_key])
        tree_walk = tree.copy()
        #print('start', first_key)
        del tree_walk[first_key]
        while len(tree_walk) > 0:
            max_fraction = (None, (-1, 1))
            if len(visited) == n:
                break
            for key in tree_walk:
                if key[0] in visited and key[1] in visited:
                    continue
                could_be_fraction = (key, (fraction[1][0] + tree_walk[key][0], fraction[1][1] + tree_walk[key][1]))
                if is_fraction_greater(could_be_fraction, max_fraction):
                    max_fraction = could_be_fraction

            next_key, next_value = max_fraction
            if next_key is not None:
                #print('next key', next_key)
                fraction = max_fraction
                visited.add(next_key[0])
                visited.add(next_key[1])
                del tree_walk[next_key]
        if is_fraction_greater(fraction, max_tree_fraction):
            max_tree_fraction = fraction

gcd_frac = gcd(max_tree_fraction[1][0], max_tree_fraction[1][1])
print('' + str(int(max_tree_fraction[1][0] / gcd_frac)) + '/' + str(int(max_tree_fraction[1][1] / gcd_frac)))
            
    
    
                
            
        
    


3/2


In [82]:
def gcd(a, b):
    if a == 0:
        return b
    else:
        return gcd(b % a, a)
    
gcd(3, 6)

3

In [ ]:

    


# print(len(trees))
    
# return_value = -1
# return_fraction = None

# trees = [{'02': (2, 4), '01': (1, 2), '12': (1, 1)}]

# for tree in trees:
#     print(tree)
#     walking_copy = tree.copy()
#     visited = set()
#     fraction = (0, 0)
#     for key, elem in tree.items():
#         print(key, visited, fraction)
#         if key[0] in visited and key[1] in visited:
#             continue
#         if (len(visited) == 0):
#             visited.add(key[0])
#             visited.add(key[1])
#             fraction = elem

#         del walking_copy[key]
        
#         if len(walking_copy) == 0:
#             continue

#         next_values = None
#         next_vertices = None
#         max_fraction = -1
#         for next_key, next_elem in walking_copy.items():
#             could_be_fraction = (fraction[0] + next_elem[0]) / (fraction[1] + next_elem[1])
#             if could_be_fraction > max_fraction:
#                 max_fraction = could_be_fraction
#                 next_vertices = next_key
#                 next_values = next_elem
#         if next_vertices:
#             visited.add(next_vertices[0])
#             visited.add(next_vertices[1])
#             del walking_copy[next_vertices]
#             fraction = (fraction[0] + next_values[0], fraction[1] + next_values[1])
#         else:
#             print('uh oh not found')
#     new_value = fraction[0] / fraction[1]
#     if new_value > return_value:
#         return_value = new_value
#         return_fraction = fraction
    
# gcd_frac = gcd(fraction[0], fraction[1])


# print('' + str(int(fraction[0] / gcd_frac)) + '/' + str(int(fraction[1] / gcd_frac)))
            